# Group 35 Python Code

## Data Collection

### Extracting PDF content from Minutes

In [1]:
#Necessary imports
import PyPDF2
import requests
import numpy as np, seaborn as sns, pandas as pd
import nltk, nltk.sentiment, sklearn
import collections
%matplotlib inline

In [2]:
#convert list to pd.series

#Preprocessing - tokenize and lower case pdf-strings
tokenizer = nltk.tokenize.TweetTokenizer()
def preprocessing(string):
    string = string.lower()
    string = tokenizer.tokenize(string)
    return string

#Apply positive/negative sets on pdf-string
#tok_doc1 = doc1_str.apply(preprocessing)

def count_dictionary(document_elem, dictionary):
    lst = []
    for word in document_elem:
        if (word in dictionary):
            lst.append(word)
    return len(lst)

In [3]:
#PDF extractor script. Creates a list separated per page.
def pdf_extractor(path):
    pdf_file = open(path, 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    c = collections.Counter(range(number_of_pages))
    res1 = []
    for i in range(len(c)):
        page = read_pdf.getPage(i)
        page_content = page.extractText()
        res1.append(page_content)
    return res1

### Creating Sentiment Variable

In [18]:
path_positive = 'https://raw.githubusercontent.com/MatPiq/SDS-Group-35/master/Exam_Project/positive.csv'
path_negative = 'https://raw.githubusercontent.com/MatPiq/SDS-Group-35/master/Exam_Project/negative.csv'
positive_cb = pd.read_csv(path_positive, sep=';', header=None)
negative_cb = pd.read_csv(path_negative, sep=';', header=None)
positive_cb = set(positive_cb[0])
negative_cb = set(negative_cb[0])


In [20]:
import os

#Sebastian file path
basedir = '/Users/matias/Documents/SDS-Group-35/Exam_Project/Minutes_PDF'

def sentiment_analysis(pos, neg, file_path):
    pos_words = []
    neg_words = []
    average_tone = []
    i = 0
    error = []
    
    for fn in os.listdir(file_path):
        try:
            print('Now treating file: ' + fn + '\nIterate: ' + str(i))
            doc = pdf_extractor(file_path + '/' + fn)
            doc = pd.Series([''.join(doc)])
            doc = doc.apply(preprocessing)

            pos_words.append(int(count_dictionary(doc[0], pos)))
            neg_words.append(int(count_dictionary(doc[0], neg)))
            average_tone.append((pos_words[i]-neg_words[i])/len(doc[0]))
            error.append(False)  
            
            i += 1
        
        except KeyError:
            error.append(True)
            pos_words.append(np.NaN)
            neg_words.append(np.NaN)
            average_tone.append(np.NaN)
            
            i += 1
            
        #date = re.findall(r'\d+', fn)
    df = pd.DataFrame(zip(pos_words, neg_words, average_tone, error))
    return df

sent_df = sentiment_analysis(positive_cb, negative_cb, basedir)


Now treating file: ppp110419e.pdf
Iterate: 0
Now treating file: ppp121217e.pdf
Iterate: 1
Now treating file: ppp010917e.pdf
Iterate: 2
Now treating file: ppp101025e.pdf
Iterate: 3


Now treating file: ppp020318e.pdf
Iterate: 4
Now treating file: ppp011015e.pdf
Iterate: 5
Now treating file: ppp171219e.pdf
Iterate: 6


Now treating file: ppp120703e.pdf
Iterate: 7
Now treating file: ppp000203e.pdf
Iterate: 8
Now treating file: ppp090210e.pdf
Iterate: 9


Now treating file: ppp160222e.pdf
Iterate: 10
Now treating file: ppp070906e.pdf
Iterate: 11


Now treating file: ppp130918e.pdf
Iterate: 12
Now treating file: ppp070619e.pdf
Iterate: 13


...

Now treating file: ppp011108e.pdf
Iterate: 123


Now treating file: ppp010426e.pdf
Iterate: 124
Now treating file: ppp181023e.pdf
Iterate: 125


Now treating file: ppp161220e.pdf
Iterate: 126
Now treating file: ppp140212e.pdf
Iterate: 127
Now treating file: ppp050127e.pdf
Iterate: 128
Now treating file: ppp111026e.pdf
Iterate: 129
Now treating file: ppp130429e.pdf
Iterate: 130
Now treating file: ppp000607e.pdf
Iterate: 131
Now treating file: ppp120215e.pdf
Iterate: 132


Now treating file: ppp180425e.pdf
Iterate: 133
Now treating file: ppp130716e.pdf
Iterate: 134


TypeError: data argument can't be an iterator

In [7]:
sent_df.columns = ['Number of positive words', 'Number of negative words', 'Average tone ratio', 'Error in processing PDF']
#print(sent_df.loc[sent_df['Number of positive words'] == True])
len(list(filter(lambda x: x > 0, sent_df['Average tone ratio'])))
sent_df

NameError: name 'sent_df' is not defined

In [ ]:
import os, re, pandas as pd, ast, datetime

#Mark file path:
path = '/Users/matias/Documents/SDS-Group-35/Exam_Project/Minutes_PDF'

files = []
files2 = []

# r=root, d=directories, f = files
for r, d, f in os.walk(basedir):
    for file in f:
        if '.pdf' in file:
            files.append('20' + file[3:5] + "-" + file[5:7] + "-" + file[7:9])
            files2.append(file)

Final_list = {'Dato':files,'Filnavn':files2}

df_files = pd.DataFrame(Final_list)
df_files['date_time'] = pd.to_datetime(df_files['Dato'])
df_files['Date time announcement'] = pd.to_datetime(df_files['Dato']) + pd.DateOffset(days=1)
df_files

In [ ]:
sent_df = pd.concat([sent_df, pd.Series(df_files['Date time announcement'])], axis = 1, join = 'inner')
sent_df

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (9,4)
plt.style.use('default')
plt.rcParams.update({'font.size': 8})
plt.rcParams['lines.markersize'] = 4

x = np.array(sent_df['Date time announcement'])
y = np.array(sent_df['Average tone ratio'])

# fit = np.polyfit(x, y, 4)
# fit_fn = poly1d(fit)
# print(fit_fn)

# x_new = np.linspace(x[0], x[-1], 50)
# y_new = f(x_new)

plt.scatter(x, y)#, 'o', x_new, y_new)
# plt.plot(x, y, fit)
plt.axhline(y = 0, color = 'r', linestyle = '-', linewidth = 0.5)
plt.show
plt.savefig('Sentiment_scatter.png')

### Collecting Stockmarket data

In [ ]:
path_omx = 'https://raw.githubusercontent.com/MatPiq/SDS-Group-35/master/Exam_Project/historic_data/omx30.csv'
cols = ['date', 'closing value']
omx = pd.read_csv(path_omx, sep=';', names = cols)

In [ ]:
#opening value column
omx['opening value'] = omx['closing value'].shift(-1)
omx['date'] = pd.to_datetime(omx['date'])


#Difference between opening and closing value
omx['dif abs'] = omx['opening value'] - omx['closing value']
omx['dif abs'] = (omx['opening value']-omx['closing value']) / omx['closing value']
omx = omx.iloc[::-1]

In [ ]:
omx.plot('date', 'closing value', grid = True, legend = False, figsize=(10, 5))
plt.savefig('omx.png')

### Collecting Repo data

In [ ]:

#import repo
path_repo = 'https://raw.githubusercontent.com/MatPiq/SDS-Group-35/master/Exam_Project/historic_data/reporate.csv'
cols = ['date', 'repoann', 'repoeff']
repo = pd.read_csv(path_repo, sep=';', error_bad_lines=False, names = cols)
repo['date'] = pd.to_datetime(repo['date'])

In [ ]:
#change in repo column
repo['repo_ch'] = repo['repoann']-repo['repoeff']

In [ ]:
repo.plot('date', 'repoann', grid = True, legend = False, figsize=(10, 5))
plt.savefig('repo.png')

### Merging complete dataframe

In [ ]:
#Necesary imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
#Merge repo and omx
repo_omx = pd.merge(repo, omx, on = 'date', how = 'inner')


In [ ]:
#Inspect and drop NaN
nan_rows = repo_omx[repo_omx.isnull().T.any().T]
repo_omx.dropna()
repo_omx.head()

In [ ]:
#merge repo_omx with sentiment analysis



### Dataframe quality assorance and validation

## Sentiment analysis

### Model validation

## Machine learning Model

### Model validation